## Documentos de texto y aprendizaje automático

Para poder aplicar técnicas de aprendizaje automático en documentos de texto es necesario transformarlos en vectores de características numéricas. Una de las formas más habituales de llevar a cabo esta tarea es mediante la representación “Bolsa de Palabras”, Bag of Words (BoW) en inglés.

Para realizar el aprendizaje del modelo BoW del texto es necesario disponer de un conjunto de documentos, conocido como corpus. Cada documento está compuesto por un conjunto de palabras. 

Para realizar la representación BoW se debe realizar lo siguiente:
* 1. Asignar un identificador entero (id) a cada palabra de cualquier documento del corpus. Es decir, al final habrá tantos id’s como palabras diferentes en el corpus. Esto se puede llevar a cabo construyendo un diccionario de palabras a índices enteros.
* 2. En cada documento #i, contar el número de ocurrencias de cada palabra, p, y almacenarlo la fila i-ésima de una matriz X
    * X[i, j] representa la característica #j, donde j es el índice (id anterior) de la palabra p en el diccionario.

Por tanto, utilizar la representación BoW implica que el número de características generado (n_features) es igual al número de palabras diferentes en el corpus. Normalmente el número de características es muy grande (>100.000). 

En resumen, BoW implica que la matriz X tiene tantas filas como documentos en el corpus y tantas columnas como número de características. Es decir, genera tantos vectores de características (ejemplos) como documentos. Si por ejemplo el número de documentos es 10.000 y el número de características generado es 100.000, almacenar X como un array de NumPy de tipo float32 (ocupa 4 bytes) requeriría de 4GB de RAM (10.000x100.000x4). Esta cantidad de memoria empieza a ser poco manejable para los ordenares convencionales de hoy en día.

Afortunadamente, muchos de los valores almacenados en X serán ceros (típicamente más del 99%) puesto que en cada documento se utilizará un subconjunto pequeño de palabras únicas que en todo el corpus. Por este motivo se dice que BoW es típicamente un conjunto de datos disperso de alta dimensionalidad (high-dimensional sparse dataset). Como consecuencia se puede ahorrar el uso de mucha memoria si solamente se almacenan las partes diferentes de cero de los vectores de características. Scikit-learn almacena la representación de BoW en matrices dispersas (scipy.sparse matrices).

Scikit-learn ofrece utilidades para abordar el problema de la extracción de características numéricas a partir de documentos de texto mediante las formas más habituales: 
* Partición de una cadena de texto y asignación de un id a cada palabra. Este proceso se suelen realizar utilizando los caracteres en blanco o los signos de puntuación como separadores de palabras. Este proceso se llama en inglés **tokenizing**, puesto que cada palabra se denomina token.
* **Conteo** del número de ocurrencias de cada palabra (token) en cada documento.
* **Normalización y ponderación** con importancia decreciente de las palabras que aparezcan en la mayoría de los documentos.

**Al proceso general de transformar una colección de documentos de texto en vectores de características numéricas se le llama vectorización. Es decir, a la estrategia compuesta por la tokenización, conteo y normalización.** Como consecuencia, los documentos están descritos por las ocurrencias de las palabras y la información relativa a las posiciones de las palabras dentro de los documentos se ignora. A continuación se muestra como se realiza cada proceso en scikit-learn.

### Tokenizing y conteo con scikit-learn

Scikit-learn ofrece una clase que implementa las etapas de **tokenizado del texto y conteo de palabras**. Esta clase se llama **CountVectorizer** y está dentro del paquete de extracción de características para texto de scikit-learn.
	
    from sklearn.feature_extraction.text import CountVectorizer

Para entender cómo funciona esta clase vamos a crear un ejemplo de un corpus formado por 4 documentos de texto en inglés. Todos los documentos sn muy cortos para visualizar mejor el proceso y en inglés ya que es el idioma por defecto en Scikit-learn y todas las funcionalidades están más desarrolladas para dicho idioma.

corpus = ['This is the first document.',
          'This is the second second document.',
          'And the third one.',
          'Is this the first document?']

Para aplicar las funcionalidades ofrecidas por CountVectorizer lo primero que hay que hacer es llamar al constructor:
	
    vectorizer = CountVectorizer()

Dicha llamada al constructor utiliza los valores por defecto de la clase. Toda la información de dicha clase se puede ver en el siguiente link: http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html#sklearn.feature_extraction.text.CountVectorizer. Algunos de los parámetros más relevantes son:
* stop_words: determina las palabras a ignorar en el proceso de tokenizado. Se puede establecer a ‘english’ que elimina las palabras más frecuentes del inglés o a una lista de palabras establecidas por el usuario. 
* max_df: número en el rango [0,1] que establece un umbral por el que si la frecuencia de una palabra es mayor que este valor se ignora en la construcción del diccionario. 
* min_df: número en el rango [0,1] que establece un umbral por el que si la frecuencia de una palabra es menor que este valor se ignora en la construcción del diccionario. 


In [1]:
# Se importa la librería CountVectorizer
from sklearn.feature_extraction.text import CountVectorizer

# Se genera el corpus, cada documento es un string y el corpus es una lista de strings (documentos)
corpus = ['This is the first document.',
          'This is the second second document.',
          'And the third one.',
          'Is this the first document?']

# Se llama al constructor de la clase CountVectorizer
vectorizer = CountVectorizer()

Una vez generado el objeto de CountVectorizer, el siguiente paso es aprender el vocabulario (diccionario) con las palabras de nuestro corpus (se realiza el proceso de tokenizing). Para ello se llama a la función fit con el corpus de documentos:

	vectorizer.fit(corpus)

Una vez aprendido el diccionario se realiza el proceso de conteo y creación de los vectores de características. Para ello se llama a la función transform pasando como argumento de entrada el corpus de documentos para realizar el conteo sobre dicho corpus:

	X = vectorizer.transform(corpus)

También existe la opción de realizar las dos operaciones anteriores de una sola vez utilizando la función fit_transform:

	X = vectorizer.fit_transform(corpus)
    
Los nombres de las características extraídas (palabras) se pueden obtener utilizando la función get_feature_names. Se pueden visualizar por pantalla:

	print vectorizer.get_feature_names()
    
Mediante la función anterior se muestran las palabras en el orden en el que han sido aprendidas. 

Realiza el aprendizaje del vocabulario a parir del corpus creado anteriormente, transfórmalo y muestra las palabras aprendidas.

In [2]:
# Se realiza el aprendizaje a partir del corpus y se transforma que los valores aprendidos
# corpusTransformado =
# BEGIN SOLUTION
corpusTransformado = vectorizer.fit_transform(corpus)
# END SOLUTION
# Se muestran los términos aprendidos (variables del modelo BoW)
# palabrasAprendidas =
# BEGIN SOLUTION
palabrasAprendidas = vectorizer.get_feature_names()
# END SOLUTION
print(palabrasAprendidas)

['and', 'document', 'first', 'is', 'one', 'second', 'the', 'third', 'this']


Para comprobar el índice de una palabra determinada, pal, en esta lista se puede utilizar la función get sobre el atributo vocabulary_:

	print vectorizer.vocabulary_.get(pal)
    
NOTA: si una palabra no está en el vocabulario aprendido el resultado es None.
    
Para ver las ocurrencias de cada característica en cada documento

	print corpusTransformado.toarray()
    
Muestra el índice de las palabras first, and y text. Muestra las ocurrencias de cada palabra.

In [3]:
# Se muestran los índices de las diferentes palabras dentro del modelo aprendido
print(vectorizer.vocabulary_.get('first'))
print(vectorizer.vocabulary_.get('and'))
print(vectorizer.vocabulary_.get('text'))
# Se muestra el modelo en forma de matriz
print(corpusTransformado.toarray())

2
0
None
[[0 1 1 1 0 0 1 0 1]
 [0 1 0 1 0 2 1 0 1]
 [1 0 0 0 1 0 1 1 0]
 [0 1 1 1 0 0 1 0 1]]


El resultado de la transformación del corpus es una matriz de tantas filas como documentos y tantas columnas como palabras diferentes haya en todos los documentos. En este caso son 4 filas y 9 columnas. Para poder conseguir visualizar el corpus transformado en forma de matriz hay que utilizar la función *toarray()* como se observa en la celda anterior.

Si se muestra directamente el corpus transformado (print corpusTransformado) se obtiene una lista de filas con el siguiente formato:
* (idDocumento, idPalabra) numOcurrencias, donde
    * idDocumento es el índice del documento
    * idPalabra es el índice de la palabra (en la lista que podéis consultar en vectorizer.get_feature_names())
    * numOcurrencias es el número de ocurrencias de la palabra idPalabra en el documento idDocumento

Ejemplo: (0,0) 3, significaría que en el primer documento, la primera palabra aprendida (mostrada en vectorizer.get_feature_names()) aparece 3 veces.

Muestra por pantalla el corpus transformado sin pasarlo a matriz.

In [9]:
# Se muestra el modelo aprendido sin forma matricial
print(corpusTransformado)

  (0, 1)	1
  (0, 2)	1
  (0, 6)	1
  (0, 3)	1
  (0, 8)	1
  (1, 5)	2
  (1, 1)	1
  (1, 6)	1
  (1, 3)	1
  (1, 8)	1
  (2, 4)	1
  (2, 7)	1
  (2, 0)	1
  (2, 6)	1
  (3, 1)	1
  (3, 2)	1
  (3, 6)	1
  (3, 3)	1
  (3, 8)	1


Para transformar un nuevo documento a esta representación, utilizando el vocabulario aprendido, se utiliza la función transform (explicada anteriormente) pasando como argumento de entrada el nuevo documento a transformar. Por ejemplo para transformar 'Something completely new.' Se ejecutaría la siguiente instrucción:

    nuevoDocT = vectorizer.transform(['Something completely new.'])

Para visualizar el resultado:

	print nuevoDocT.toarray()

El resultado será una lista con 9 ceros puesto que ninguna de las palabras de la frase ha sido aprendida mediante el corpus introducido. Hay 9 ceros puesto que una vez aprendido el vocabulario no se incluyen nuevas palabras a menos que se entrene de nuevo con nuevas palabras (un nuevo corpus).

Realiza las dos instrucciones anteriores.  El resultado debe ser una lista con todo ceros.

In [4]:
# Se realiza la transformación de un nuevo documento (string) en base al corpus aprendido
# nuevoDocT =
# BEGIN SOLUTION
nuevoDocT = vectorizer.transform(['Something completely new.'])
# END SOLUTION
print(nuevoDocT.toarray())

[[0 0 0 0 0 0 0 0 0]]


Crea un nuevo objeto de la clase CountVectorizer pero esta vez establece como stop-words la lista con las siguientes palabras: this, is, the, and.

Aprende el vocabulario a partir del corpus inicial y muestra tanto la lista de palabras aprendidas como el corpus transformado por el nuevo vocabulario. Observa las diferencias de la transformación al utilizar esta lista de stop-words o sin usarlas.

In [5]:
# Se genera otro modelo utilizando una lista de palabras como stop-words (constructor, aprendizaje y transformación)
    # No se tienen en cuenta como posibles variables del modelo
# vectorizer1 = CountVectorizer(#<RELLENAR>)
# corpusTransformado1 = #<RELLENAR>
# BEGIN SOLUTION
vectorizer1 = CountVectorizer(stop_words=['this','is','the','and'])
corpusTransformado1 = vectorizer1.fit_transform(corpus)
# END SOLUTION
# Se muestran las variables del nuevo modelo y el modelo en forma matricial
# palabrasAprendidas1 =
# BEGIN SOLUTION
palabrasAprendidas1 = vectorizer1.get_feature_names()
# END SOLUTION
print(palabrasAprendidas1)
print(corpusTransformado1.toarray())

['document', 'first', 'one', 'second', 'third']
[[1 1 0 0 0]
 [1 0 0 2 0]
 [0 0 1 0 1]
 [1 1 0 0 0]]


Crea un nuevo objeto de la clase CountVectorizer pero esta vez establece como stop-words las del idioma inglés. Vuelve a mostrar tanto la lista de palabras aprendidas como el corpus transformado.

In [6]:
# Se genera otro modelo utilizando la lista de stop-words del inglés (constructor, aprendizaje y transformación)
    # No se tienen en cuenta como posibles variables del modelo
# vectorizer2 = CountVectorizer(#<RELLENAR>)
# corpusTransformado2 = #<RELLENAR>
# BEGIN SOLUTION
vectorizer2 = CountVectorizer(stop_words='english')
corpusTransformado2 = vectorizer2.fit_transform(corpus)
# END SOLUTION

# Se muestran las variables del nuevo modelo y el modelo en forma matricial
# palabrasAprendidas2 =
# BEGIN SOLUTION
palabrasAprendidas2 = vectorizer2.get_feature_names()
# END SOLUTION
print(palabrasAprendidas2)
print(corpusTransformado2.toarray())

['document', 'second']
[[1 0]
 [1 2]
 [0 0]
 [1 0]]


### De ocurrencias a frecuencias

El conteo de ocurrencias de las palabras es un buen punto de inicio pero hay un problema: los documentos largos tendrán conteos más grandes que los cortos (incluso si tratan el mismo tema).
Para evitar este problema basta con dividir el número de ocurrencias de cada palabra en un documento por el número total de palabras del documento. A estas nuevas características se les llama frecuencias (Term Frequencies, TF, en inglés).
Posteriormente, las frecuencias se suelen refinar de tal modo que la importancia de las palabras que aparezcan en muchos documentos del corpus sea disminuida. El motivo es que las palabras que aparecen en muchos documentos del corpus (como preposiciones por ejemplo) son menos informativas que las que aparecen en una porción pequeña de ellos. A este proceso de refinado se le llama TF-IDF: Term Frequency times Inverse Document Frequency. Es decir, se multiplica la frecuencia, TF, por la inversa de la frecuencia en todos los documentos (IDF por sus siglas en inglés):

   $tf-idf(t,d) = tf(t)*idf(t)$
 
donde t es la palabra a normalizar y d es el documento en el que se realiza la normalización. La parte correspondiente a la inversa de la frecuencia del documento se calcula como

   $idf(t) = log(\frac{1+n_d}{1+df(d,t)})$
 
donde $n_d$ es el número total de documentos en el corpus y $df(d,t)$ es el número de documentos que contienen la palabra $t$. 
Finalmente, una vez realizado el cálculo de TF-IDF para todas las palabras de un documento, los valores resultantes de cada documento se normalizan por la norma Euclídea

  $w_{norm}=\frac{v}{||v||_2}=\frac{v}{\sqrt{(v_1^2+...+v_n^2)}}$

Scikit-learn ofrece una clase que realiza todos estos cálculos a partir de una matriz de conteos (la devuelta por la clase CountVectorizer). Esta clase se llama **TfidfTransformer** y está dentro del paquete de extracción de características para documentos de texto.

	from sklearn.feature_extraction.text import TfidfTransformer

Para poder utilizar todas las funciones de dicha clase lo primero que se debe hacer es crear un objeto de dicha clase. Toda la información de esta clase se puede consultar en la siguiente web: http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfTransformer.html#sklearn.feature_extraction.text.TfidfTransformer. Los valores por defecto del constructor realizan el proceso explicado anteriormente:

	normalizar = TfidfTransformer()

Una vez creado el objeto de la clase lo primero que se debe realizar es aprender de los datos del conteo para poder realizar las normalizaciones. Para ello se utiliza la función fit utilizando como variable de entrada una matriz de conteos (obtenida con CountVectorizer)
	
	normalizar.fit(matrizConteo)

Una vez realizado el aprendizaje se puede utilizar el objeto para llevar a cabo la obtención de los pesos correspondientes a cada palabra a partir de la matriz de conteos. Para ello se utiliza la función transform

	BoW = normalizar.transform(matrizConteo)

Las dos funciones anteriores se pueden realizar de una vez utilizando la función fit_transform

	BoW = normalizar.fit_transform(matrizConteo)
    
Para mostrar el modelo BoW aprendido tras realizar todas las operaciones podemos utilizar la función toarray() del objeto tal y como hemos realizado con el objeto de la clase CountVectorizer.

Realiza las instrucciones anteriores y muestra el modelo BoW obtenido para la configuración por defecto de la clase CountVectorizer (variable corpusTransformado).

In [10]:
# Se importa la librería en la que está la clase TfidfTransformer
from sklearn.feature_extraction.text import TfidfTransformer
# Se llama al constructor de la clase TfidfTransformer
# tdidf =
# BEGIN SOLUTION
tdidf = TfidfTransformer()
# END SOLUTION
# Se realiza el aprendizaje de los pesos y la transformación del corpus en base a la matriz de conteos obtenida anteriormente
    # Utilizar la matriz de conteos obtenida por CountVectorizer con los parámetros por defecto
# BoW =
# BEGIN SOLUTION
BoW = tdidf.fit_transform(corpusTransformado)
# END SOLUTION
# Se muestra el modelo BoW aprendido en forma matricial
print(BoW.toarray())

[[0.         0.43877674 0.54197657 0.43877674 0.         0.
  0.35872874 0.         0.43877674]
 [0.         0.27230147 0.         0.27230147 0.         0.85322574
  0.22262429 0.         0.27230147]
 [0.55280532 0.         0.         0.         0.55280532 0.
  0.28847675 0.55280532 0.        ]
 [0.         0.43877674 0.54197657 0.43877674 0.         0.
  0.35872874 0.         0.43877674]]


### Aprendizaje de un clasificador a partir de la representación BoW

La representación BoW crea por cada documento de texto un ejemplo. Es decir, un vector de características (valores TF-IDF). Por tanto, todos los documentos de texto estarán representados por un vector con el mismo número de características. Como resultado, es posible realizar el aprendizaje de un clasificador utilizando la representación BoW de cada documento junto con la clase a la que pertenece dicho documento. Las clases de los documentos serán diferentes según el problema a abordar como la categoría del texto (deporte, economía, política, etc..), el sentimiento del texto (opinión positiva, negativa o neutra), entre otros problemas que se pueden afrontar.

Para resolver este tipo de problemas las etapas a realizar son las siguientes:
* Leer los ejemplos de entrenamiento del problema y formar el corpus.
* Realizar el tokenizado y conteo del corpus.
* Realizar la transformación a frecuencias del conteo obtenido.
* Aprender el clasificador (cualquier clasificador) con los datos resultantes.
* Leer los ejemplos de test del problema y formar el corpus de test.
* Realizar el tokenizado y conteo de este corpus utilizando lo aprendido con el corpus de entrenamiento.
* Realizar la transformación a frecuencias del nuevo conteo con lo aprendido con el corpus de entrenamiento.
* Predecir la clase de los documentos utilizando el clasificador aprendido.
* Calcular el porcentaje de acierto.

## Práctica

Para realizar esta práctica vamos a utilizar un dataset que contiene documentos de texto ofrecido por scikit_learn. Este dataset se llama Twenty Newsgroups y su información se puede consultar en la URL: http://scikit-learn.org/stable/modules/generated/sklearn.datasets.fetch_20newsgroups.html#sklearn.datasets.fetch_20newsgroups. Su descripción oficial es la siguiente: 

    "The 20 Newsgroups data set is a collection of approximately 20,000 newsgroup documents, partitioned (nearly) evenly across 20 different newsgroups. To the best of our knowledge, it was originally collected by Ken Lang, probably for his paper “Newsweeder: Learning to filter netnews,” though he does not explicitly mention this collection. The 20 newsgroups collection has become a popular data set for experiments in text applications of machine learning techniques, such as text classification and text clustering."

Para trabajar con este dataset lo primero que debemos hacer es importarlo:

    from sklearn.datasets import fetch_20newsgroups

Una vez importado se leerán los datos de los documentos. Para ello, la llamada al constructor de la clase es la siguiente:, se establece como subset la opción train y de esta forma devuelve los datos de entrenamiento (poner esta opción a test para leer los datos de test).

    twenty_train = fetch_20newsgroups(subset=tipoDatos, shuffle=aleatorio, random_state=semilla, categories=clasesDocumentos)

Los parámetros son los siguientes:
* tipoDatos: string que determina si los datos son de entrenamiento (asignar 'train') o de test (asignar 'test')
* aleatorio: valor booleano que establece si se aleatorizan los datos o no.
* semilla: valor entero que determina la semilla para la generación de números aleatorios. De esta forma los experimentos serán reproducibles.
* clasesDocumentos: lista con los nombre de las clases de los docuemntos a leer. Si se asigna a None se leen los documentos de todas las clases.

El objeto generado (variable twenty_train) tiene la misma estructura que todos los datasets con los que hemos trabajado. Por tanto:
* Los **datos de entrada** correspondientes a los documentos de texto se encuentran en el campo **data**. Por ejemplo, si se desea mostrar el primer documento se puede ejecutar:	print("\n".join(twenty_train.data[0].split("\n")))
* Los **nombres de las clases** se encuentran en el campo **target_names**. En este corpus de documentos se encuentran documentos clasificados en 20 clases diferentes, para visualizarlos ejecuta: print twenty_train.target_names
* Las **clases** de cada documento se encuentran en el campo **target**. Por ejemplo si se quiere obtener la clase del primer documento se ejecutaría: print twenty_train.target[0]
* Si anidamos los campos target_names y target podemos visualizar la clase de cada documento. Por ejemplo si se desea conocer la clase del primer documento se ejecutaría: print twenty_train.target_names[twenty_train.target[0]]

Ejercicio 1: lee los datos de entrenamiento (utiliza el valor 42 como semilla para la lectura) correspondientes a las clases de nombre 'alt.atheism', 'soc.religion.christian', 'comp.graphics', 'sci.med'. Muestra el contenido del primer documento, su clase y el nombre de dicha clase.

In [11]:
# Se importa el dataset fetch_20newsgroups
from sklearn.datasets import fetch_20newsgroups

# se realiza la lectura de los datos de entrenamiento: poner el parámetro shuffle a True y utilizar las clases apropiadas
# clases = [#<RELLENAR>]
# twenty_train = fetch_20newsgroups(#<RELLENAR>)
# BEGIN SOLUTION
clases = ['alt.atheism', 'soc.religion.christian', 'comp.graphics', 'sci.med']
twenty_train = fetch_20newsgroups(subset='train', shuffle=True, categories=clases, random_state=42)
# END SOLUTION

# Se muestra el primer documento (mail) junto con su clase y el nombre de la misma
print("\n".join(twenty_train.data[0].split("\n")))
print(twenty_train.target[0])
print(twenty_train.target_names[twenty_train.target[0]])

From: sd345@city.ac.uk (Michael Collier)
Subject: Converting images to HP LaserJet III?
Nntp-Posting-Host: hampton
Organization: The City University
Lines: 14

Does anyone know of a good way (standard PC application/PD utility) to
convert tif/img/tga files into LaserJet III format.  We would also like to
do the same, converting to HPGL (HP plotter) files.

Please email any response.

Is this the correct group?

Thanks in advance.  Michael.
-- 
Michael Collier (Programmer)                 The Computer Unit,
Email: M.P.Collier@uk.ac.city                The City University,
Tel: 071 477-8000 x3769                      London,
Fax: 071 477-8565                            EC1V 0HB.

1
comp.graphics


Ejercicio 2: Realiza el proceso de tokenizado, conteo y obtención de los pesos (td-idf) del corpus generado (el resultado de la lectura).

In [12]:
# Se crea el objeto de la clase CountVectorizer
count_vect = CountVectorizer()
# Realizar el aprendizaje y la transformación a partir de los datos de entrenamiento leídos anteriormente
# X_train_counts =
# BEGIN SOLUTION
X_train_counts = count_vect.fit_transform(twenty_train.data)
# END SOLUTION

# Se crea el objeto de la clase TfidfTransformer
tfidf_transformer = TfidfTransformer()
# Realizar el aprendizaje y la transformación a partir de la matriz de conteo generada anteriormente
# X_train_tfidf =
# BEGIN SOLUTION
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
# END SOLUTION
# Se muestran las dimensiones de la matriz que representa al modelo BoW aprendido
print(X_train_tfidf.shape)

(2257, 35788)


Una vez realizado el proceso anterior se va a realizar el aprendizaje de un clasificador. En este caso vamos a utilizar la versión multinomial de **Naïve Bayes** ya que en la literatura clásica se suele utilizar este clasificador en conjunción con el modelo BoW para afontar problemas de minería de textos. Pese a que no lo hemos visto en esta asignatura, el clasificador Naïve Bayes es un clasificador bastante sencillo que se basa en el teorema de Bayes para realizar el aprendizaje del modelo de clasificación. La librería Scikit-learn nos ofrece dicho clasificador para poder utlizarlo fácilmente ya que nos bastará con invocar su constructor, su método de entrenamiento (*fit*) y su método de predicción de la clase de nuevos ejemplos (*predict*). Para utilizar este clasificador primero hay que importarlo

    from sklearn.naive_bayes import MultinomialNB
    
Toda la información de esta clase la podéis encontrar en la siguiente URL: http://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.MultinomialNB.html#sklearn.naive_bayes.MultinomialNB. Los parámetros más importantes de esta clase son:
* alpha: valor real que determina la constante para realizar el suavizado de Laplace. Por defecto el valor es 1.0. Si no se quiere realizar suavizado asignar el valor 0.0.
* fit_prior: valor booleano que determina si se aprenden las probabilidades a priori de las clases o no. El valor por defecto es True.

Ejercicio 3: Crear un objeto de MultinomialNB con los valores por defecto. Realizar el aprendizaje (fit) con el modelo BoW obtenido en el ejercicio 2 y las clases correspondientes a los documentos.

In [13]:
# Se importa la librería de la clase Naïve Bayes
from sklearn.naive_bayes import MultinomialNB

# Se realiza el aprendizaje del clasificador Naïve Bayes
# clf_NB =
# BEGIN SOLUTION
clf_NB = MultinomialNB().fit(X_train_tfidf, twenty_train.target)
# END SOLUTION

Ejercicio 4: Leer los datos de test utilizando el valor 42 como semilla. Realizar el proceso de tokenizado, conteo y obtención de los pesos de cada palabra (td-idf) del corpus generado con los parámetros aprendidos en el ejercicio 2. Realizar la predicción de los ejemplos con el calificador aprendido en el ejercicio 3. Obtener su porcentaje de acierto.

In [14]:
# Se importa la librería metrics para calcular el rendimiento de los clasificadores
from sklearn import metrics

# Se leen los datos de test: asignar el parámetro shuffle a True y utilizar las clases apropiadas
# twenty_test = 
# BEGIN SOLUTION
twenty_test = fetch_20newsgroups(subset='test', shuffle=True, categories=clases, random_state=42)
# END SOLUTION

# Realizar la transformación de los documentos de test tanto con CountVectorizer como con TfidfTransformer
# X_test_counts =
# X_test_tfidf =
# BEGIN SOLUTION
X_test_counts = count_vect.transform(twenty_test.data)
X_test_tfidf = tfidf_transformer.transform(X_test_counts)
# END SOLUTION

# Realizar la predicción de los datos de test utiliando el clasificador Naïve Bayes aprendido anteriormente
# prediccionNB_test =
# BEGIN SOLUTION
prediccionNB_test = clf_NB.predict(X_test_tfidf)
# END SOLUTION
# Se calcula e imprime el porcentaje de acierto (entre 0 y 100)
# accuracy =
# BEGIN SOLUTION
accuracy = metrics.accuracy_score(prediccionNB_test, twenty_test.target)*100.0
# END SOLUTION
print (accuracy)

83.48868175765645


Ejercicio 5: crear una Pipeline compuesta por los 3 objetos necesarios para resolver el problema de predecir la clase de los documentos de texto (CountVectorizer, TfidfTransformer y MultinomialNB). Entrenarla con los datos de entrenamiento, predecir las clases de los datos de test y obtener su porcentaje de acierto.

In [15]:
# Se importa la librería para utilizar la clase Pipeline
from sklearn.pipeline import Pipeline

# Se crea la pipeline con las 3 fases necesarias para solventar el problema
# text_clf =
# BEGIN SOLUTION
text_clf = Pipeline([('vect', CountVectorizer()), ('tfidf', TfidfTransformer()), ('clf', MultinomialNB())])
# END SOLUTION
# Se realiza el aprendizaje de todos los objetos de la pipeline
# text_clf =
# BEGIN SOLUTION
text_clf = text_clf.fit(twenty_train.data, twenty_train.target)
# END SOLUTION

# Se realiza la predicción de los datos de test
# prediccionPipeline_test =
# BEGIN SOLUTION
prediccionPipeline_test = text_clf.predict(twenty_test.data)
# END SOLUTION
# Se calcula e imprime el porcentaje de acierto (entre 0 y 100)
# accuracyPipeline =
# BEGIN SOLUTION
accuracyPipeline = metrics.accuracy_score(prediccionPipeline_test, twenty_test.target)*100.0
# END SOLUTION
print (accuracyPipeline)

83.48868175765645


Ejercicio 6: utiliza la función *GridSearchCV* de la librería *model_selection* para buscar la configuración óptima del clasificador compuesto aplicando la validación cruzada de 10 particiones. Como el proceso es muy costoso si utilizamos todos los ejemplos de entrenamiento se debe entrenar con los 500 primeros ejemplos. Los parámetros y valores a optimizar son:
* CountVectorizer
    * stop_words: ‘english’, None
* TfidfTransformer
    * use_idf: True, False
* MultinomialNB 
    * alpha: 0.5, 1, 2
    * fit_prior: True, Flase
    
Fijad la semilla de Numpy a 12.

In [16]:
# Se importa la librería para poder hacer la selección de los valores de los parámetros con validación cruzada
from sklearn import model_selection
import numpy as np
# Se crea la pipeline con las 3 fases necesarias para solventar el problema
# text_clf_pip =
# BEGIN SOLUTION
text_clf_pip = Pipeline([('vect', CountVectorizer()), ('tfidf', TfidfTransformer()), ('clf', MultinomialNB())])
# END SOLUTION
# Se crea el grid de parámetros (diccionario)
# parameters_NB =
# BEGIN SOLUTION
parameters_NB = {'vect__stop_words': ['english', None],
              'tfidf__use_idf': (True, False),
              'clf__alpha': (0.5, 1.0, 2.0),
              'clf__fit_prior': (True, False)}
# END SOLUTION

# Se llama al constructor de la clase GridSearchCV
# gs_clf_nb =
# BEGIN SOLUTION
np.random.seed(12)
gs_clf_nb = model_selection.GridSearchCV(text_clf_pip, parameters_NB, cv=10, n_jobs=1, return_train_score=True)
# END SOLUTION
# Se realiza el aprendizaje utilizando la clase GridSearchCV con los 500 primeros documentos de train
# gs_clf_nb =
# BEGIN SOLUTION
gs_clf_nb = gs_clf_nb.fit(twenty_train.data[:500], twenty_train.target[:500])
# END SOLUTION

# Se imprime el mejor porcentaje de acierto y los resultados de todas las configuraciones
# BEGIN SOLUTION
print (gs_clf_nb.best_score_)
# Se imprimen los valores de los parámetros de la mejor configuración
resultadosMostrar = zip(gs_clf_nb.cv_results_['params'],gs_clf_nb.cv_results_['mean_test_score'],gs_clf_nb.cv_results_['mean_train_score'])
for params, mean_test_score, mean_train_score in resultadosMostrar:
    print("%0.3f (Train: %0.3f) for %r" % (mean_test_score, mean_train_score, params))
print()
# END SOLUTION

0.934
0.922 (Train: 0.999) for {'clf__alpha': 0.5, 'clf__fit_prior': True, 'tfidf__use_idf': True, 'vect__stop_words': 'english'}
0.808 (Train: 0.971) for {'clf__alpha': 0.5, 'clf__fit_prior': True, 'tfidf__use_idf': True, 'vect__stop_words': None}
0.902 (Train: 0.985) for {'clf__alpha': 0.5, 'clf__fit_prior': True, 'tfidf__use_idf': False, 'vect__stop_words': 'english'}
0.666 (Train: 0.829) for {'clf__alpha': 0.5, 'clf__fit_prior': True, 'tfidf__use_idf': False, 'vect__stop_words': None}
0.934 (Train: 1.000) for {'clf__alpha': 0.5, 'clf__fit_prior': False, 'tfidf__use_idf': True, 'vect__stop_words': 'english'}
0.826 (Train: 0.984) for {'clf__alpha': 0.5, 'clf__fit_prior': False, 'tfidf__use_idf': True, 'vect__stop_words': None}
0.916 (Train: 0.993) for {'clf__alpha': 0.5, 'clf__fit_prior': False, 'tfidf__use_idf': False, 'vect__stop_words': 'english'}
0.706 (Train: 0.858) for {'clf__alpha': 0.5, 'clf__fit_prior': False, 'tfidf__use_idf': False, 'vect__stop_words': None}
0.896 (Train: 

C:\Users\joseantonio.sanz.DYC\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\model_selection\_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


Ejercicio 7. Utiliza el clasificador compuesto optimizado aprendido en el ejercicio anterior para realizar la predicción de los documentos de test y obtén su accuracy.

In [17]:
# Se realiza la predicción de los datos de test con la mejor configuración encontrada anteriormente
# predictedPip_NB_test =
# BEGIN SOLUTION
predictedPip_NB_test = gs_clf_nb.predict(twenty_test.data)
# END SOLUTION
# Se calcula e imprime el porcentaje de acierto (entre 0 y 100)
# accuracyPipeline_NB_opt =
# BEGIN SOLUTION
accuracyPipeline_NB_opt = metrics.accuracy_score(predictedPip_NB_test, twenty_test.target)*100.0
# END SOLUTION
print (accuracyPipeline_NB_opt)

88.34886817576565


Las máquinas de soporte vectorial (en inglés Support Vector MAchines, SVMs) son uno de los clasificadores más precisos de la actualidad. Pese a que no hemos visto este clasificador en esta asigantura lo vamos a utilizar para comprobar la potencia que ofrece la combinación entre el modelo BoW y el clasificador Naïve Bayes que es mucho más simple que una SVM. La librería Scikit-learn nos ofrece una gran facilidad para utlizar los clasificadores aunque no conozcamos qué es lo que hacen para construir el modelo o para clasificar nuevos ejemplos. Para utilizar las SVMs lo primero que debemos hacer es importar la SVM que está dentro de la librería svm. 

    from sklearn import svm
    
El constructor de dicho clasificador es SVC y todos sus detalles se pueden encontrar en la URL: http://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html#sklearn.svm.SVC. Los parámetros más importantes de esta clase son:
* C: valor real que determina el peso asignado a la penalización de los errores (slack variables).
* kernel: string que determina el tipo de kernel a utilizar. Puede tomar los valores: 'linear', 'poly', 'rbf' y 'sigmoid'.
* Existen varios parámetros más que determinan los valores de los diferentes tipos de kernel. Algunos de ellos son los parámetros degree (para el polinomial), gamma (para todos menos para el kernel lineal) o coef0 (para el polinomial y sigmoideo).

Ejercicio 8: Vamos a replicar el experimento de los ejercicios 6 y 7 pero utilizando una SVM. 

Los parámetros y valores a optimizar de las tres componentes de la Pipeline son:
* CountVectorizer
    * Stop_words: ‘english’, None
* TfidfTransformer
    * Use_idf: True, False
* SVC
    * C: 0.1, 1.0 y 10.0
    * kernel: 'linear', y 'rbf'

In [18]:
# Se importa la librería para poder utilizar la clase SCV (clasificador de SVM)
from sklearn import svm

# Se crea la pipeline con las 3 fases necesarias para solventar el problema
# text_clf_svc =
# BEGIN SOLUTION
text_clf_svc = Pipeline([('vect', CountVectorizer()), ('tfidf', TfidfTransformer()), ('clf', svm.SVC())])
# END SOLUTION
# Se crea el grid de parámetros
# parameters_svc =
# BEGIN SOLUTION
parameters_svc = {'vect__stop_words': ['english', None],
              'tfidf__use_idf': (True, False),
              'clf__C': (0.1, 1.0, 10.0),
              'clf__kernel': ('linear', 'rbf')}
# END SOLUTION

# Se llama al constructor de la clase GridSearchCV
# gs_clf_svm =
# BEGIN SOLUTION
np.random.seed(12)
gs_clf_svm = model_selection.GridSearchCV(text_clf_svc, parameters_svc, cv=10, n_jobs=1, return_train_score=True)
# END SOLUTION
# Se realiza el aprendizaje utilizando la clase GridSearchCV con los 500 primeros documentos de train
# gs_clf_svm =
# BEGIN SOLUTION
gs_clf_svm = gs_clf_svm.fit(twenty_train.data[:500], twenty_train.target[:500])
# END SOLUTION

# Se imprime el mejor porcentaje de acierto y los resultados de todas las configuraciones
# BEGIN SOLUTION
print (gs_clf_svm.best_score_) 
# Se imprimen los valores de los parámetros de la mejor configuración
resultadosMostrar = zip(gs_clf_svm.cv_results_['params'],gs_clf_svm.cv_results_['mean_test_score'],gs_clf_svm.cv_results_['mean_train_score'])
for params, mean_test_score, mean_train_score in resultadosMostrar:
    print("%0.3f (Train: %0.3f) for %r" % (mean_test_score, mean_train_score, params))
print()
# END SOLUTION

C:\Users\joseantonio.sanz.DYC\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
C:\Users\joseantonio.sanz.DYC\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\model_selection\_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


0.936
0.444 (Train: 0.490) for {'clf__C': 0.1, 'clf__kernel': 'linear', 'tfidf__use_idf': True, 'vect__stop_words': 'english'}
0.454 (Train: 0.472) for {'clf__C': 0.1, 'clf__kernel': 'linear', 'tfidf__use_idf': True, 'vect__stop_words': None}
0.520 (Train: 0.545) for {'clf__C': 0.1, 'clf__kernel': 'linear', 'tfidf__use_idf': False, 'vect__stop_words': 'english'}
0.450 (Train: 0.482) for {'clf__C': 0.1, 'clf__kernel': 'linear', 'tfidf__use_idf': False, 'vect__stop_words': None}
0.302 (Train: 0.303) for {'clf__C': 0.1, 'clf__kernel': 'rbf', 'tfidf__use_idf': True, 'vect__stop_words': 'english'}
0.294 (Train: 0.298) for {'clf__C': 0.1, 'clf__kernel': 'rbf', 'tfidf__use_idf': True, 'vect__stop_words': None}
0.298 (Train: 0.301) for {'clf__C': 0.1, 'clf__kernel': 'rbf', 'tfidf__use_idf': False, 'vect__stop_words': 'english'}
0.292 (Train: 0.291) for {'clf__C': 0.1, 'clf__kernel': 'rbf', 'tfidf__use_idf': False, 'vect__stop_words': None}
0.932 (Train: 1.000) for {'clf__C': 1.0, 'clf__kernel'

Ejercicio 9. Utiliza el clasificador compuesto optimizado aprendido en el ejercicio anterior para realizar la predicción de los documentos de test y obtén su accuracy.

In [19]:
# Se realiza la predicción de los datos de test con la mejor configuración encontrada anteriormente
# predictedPip_SVC_test =
# BEGIN SOLUTION
predictedPip_SVC_test = gs_clf_svm.predict(twenty_test.data)
# END SOLUTION
# Se calcula e imprime el porcentaje de acierto (entre 0 y 100)
# accuracyPipeline_SVC_opt =
# BEGIN SOLUTION
accuracyPipeline_SVC_opt = metrics.accuracy_score(predictedPip_SVC_test, twenty_test.target)*100.0
# END SOLUTION
print (accuracyPipeline_SVC_opt)

90.21304926764314
